In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Decision trees
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#import graphviz
import os

In [4]:
# Set up the data that we want to create a DT on.
# import the fx data , econ and value data for EURUSD.
# then create the features (on trend and econ data) standardise and run a DT on the x_train sample.
# what is  target? 1 day ahead or long days ahead? trade on binary data.
csv_file = {"FXData" : r"C:\Users\edgil\Documents\Masters\dissertation\data\CurrencyData.csv",
            "ValueData" : r"",
            "EconData" : r"",
            }
fxdata = pd.read_csv(csv_file["FXData"])

In [6]:
# use this to make sure the dates are imported correctly as UK dates
fxdata['Date'] = pd.to_datetime(fxdata['Date'], format= '%d/%m/%Y %H:%M')
# Separate out the EURUSD factor
eurusd = fxdata[["Date", "EURUSD"]]


In [7]:
# Build out the featur set on price, this may need to be created using funcional process.
eurusd["logret"] = np.log(eurusd["EURUSD"]) - np.log(eurusd["EURUSD"].shift(1))
# Standardising the daily rets and accumulating the standardised returns, or should we sum the % ret and standardise by its own history
# is difference between different accumulated retusn horizons the same as the macd?
# should we standardise by the 1 year forward vol?
targetlkbk = 5
lkbk1M = 22
lkbk3M = 66
lkbk6M = 132
lkbk9M = 198
eurusd['1MRet'] = eurusd["logret"].rolling(lkbk1M).sum()
eurusd['3MRet'] = eurusd["logret"].rolling(lkbk3M).sum()
eurusd['6MRet'] = eurusd["logret"].rolling(lkbk6M).sum()
eurusd['9MRet'] = eurusd["logret"].rolling(lkbk9M).sum()
eurusd['1Mv3MRet'] = eurusd["1MRet"].sub(eurusd["3MRet"])
eurusd['1Mv6MRet'] = eurusd["1MRet"].sub(eurusd["6MRet"])
# create a target vector to train on.
# must think deeply about what this will look like
# to start, this is a 1M forard return calculation, is it right to use overlapping 1M fwd rets? it seems not...
eurusd["target"] = eurusd['logret'].iloc[::-1].rolling(targetlkbk).sum().values[::-1]
eurusd['target_binary'] = eurusd['target'].apply(np.sign) 

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [58]:
# Very important step is to truncate the data so that we do not see the last 1 year of data.
# Q. should we not have a rollign window type of model? or just always aggregate the data from the start?
# how ong is testing? we should we the train and test to sizes which make sense to the type of model we use going forward.
eurusd = eurusd.loc[eurusd['Date'] < "2018-01-01 00:00"]
# train size should be at least 5 years?
eurusd_train = eurusd.loc[eurusd['Date'] < "2003-01-01 00:00"]
eurusd_test = eurusd.loc[eurusd['Date'] > "2010-02-01 00:00"]
# create a target vector to train on.
# must think deeply about what this will look like



In [60]:
# Standardise the data using sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# standardise the data now.
# remove dates
data_cols = [col for col in eurusd_train.columns if col not in ["Date", "target", "target_binary"]]
data_train = eurusd_train.loc[:, data_cols]
data_test = eurusd_test.loc[:, data_cols]
eurusd_train_normed = pd.DataFrame(scaler.fit_transform(data_train), columns = list(data_train.columns))
eurusd_test_normed = pd.DataFrame(scaler.transform(data_test), columns = list(data_test.columns)) 
#print(eurusd_train_normed.tail(50))
eurusd_train_normed.shape

(18768, 8)

In [61]:
#### This is where we choose and set up the models!
# need to re do the models here.
X = eurusd_train_normed[['1Mv3MRet', "1Mv6MRet"]]
Y = eurusd_train["target_binary"]
X_test = eurusd_test_normed
Y_test = eurusd_test["target_binary"]
# clean the data and nan values
X = X.replace(np.nan, 0)
Y = Y.replace(np.nan, 0)
X_test = X_test.replace(np.nan, 0)
Y_test = Y_test.replace(np.nan, 0)
#RF = RandomForestClassifier(n_estimators = 150, max_features = 5)
#RF.fit(X, Y)
clf = tree.DecisionTreeClassifier(max_leaf_nodes = 6, max_depth = 8)
clf = clf.fit(X, Y)
#tree.export_graphviz(clf, out_file='tree.dot')

In [62]:
# visualise the data
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
"""export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
                """
tree.export_graphviz(clf, out_file=dot_data, class_names=['Sell',"Hold","Buy"]
                     , filled=True, rounded=True, special_characters = True) 
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("treey.pdf") 
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

True

In [1]:
# Rough work
#eurusd = eurusd.replace(np.nan, 0)
#print(eurusd.tail(25))
#eurusd["target"] = eurusd['logret'].iloc[::-1].rolling(2).sum().values[::-1]
#print(eurusd.head(10))
#data_cols = [col for col in eurusd_train.columns if col not in ["Date", "target", "target_binary"]]
#print(data_cols)
from graphviz import Digraph

dot = Digraph(comment="The round table")
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')
dot.render('round-table.gv', view=True)

'round-table.gv.pdf'

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
data_file = pd.read_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyData.csv") # pd.read_csv(r"/storage/eurusd_train_normed.csv")
data_file = data_file.replace(np.nan, 0)
#eurusd_test = eurusd_test.replace(np.nan, 0)
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
ntree = 150 # [21, 66]
trade_horizon = 24
model_features = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff",
                   "MF_ema_diff","LF_ema_diff", "LDN", "NY", "Asia", "target"]
test_buffer = 5
data_size = 50000
test_split = 0.75
# signal threshold, when using classifier
thold = 0.55
total_data_needed = int(data_size*(1 + test_split)) + test_buffer
###### Set Targets ##############
trade_horizon = 24 # in hours
use_risk_adjusted = False # if True: training on the sharpe return else raw
use_binary = False # set to true if you are using the risk adjusted and want it binary
use_classifier = True
data_file["target"] = calculate_target(data_file,trade_horizon,use_risk_adjusted)
################### Standardise Entire Dataset using rolling lookback windows ###############
features_to_standardise = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff",
                                "MF_ema_diff","LF_ema_diff"]
# roughly 3 yrs of data slightly less actually
window = 15000
data_normed = standardise_data(data_file,model_features ,features_to_standardise, window)
# add extra features non standardised
data_normed['Date'] = data_file['Date'].iloc[window:]
data_normed['CCY'] = data_file['CCY'].iloc[window:]
#data_normed = data_normed.replace(np.nan, 0)
start_row = 0
use_separated_chunk = False # Use a rollign window to train and test
################ Loop through the full dataset in terms of the training and testing.
while start_row < data_file.shape[0]:
    # first check if there is enough data left
    if (start_row + total_data_needed) > data_file.shape[0]:
        # if we are about to go over the limit, then just return the last data_size + test size proportion of data
        trunc_data = data_file.iloc[-total_data_needed:,:]
    # we need to increment over the data size
    if use_separated_chunk:
        # this means we jump across the full previous train and test data
        trunc_data = data_file.loc[start_row:,:]
        start_row += total_data_needed
    else:
        # this rools the data so that the new training will overlap on the old test set and create a new separated test set
        trunc_data = data_file.loc[start_row:,:]
        start_row += data_size
    # standardise the data
    #################### Set up training and testing ########################
    print("start_row %s" % start_row)
    trunc_data.to_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\TruncData_%s.csv" % str(start_row)) 
    # create data_set
    train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer)
    train_sample = train[model_features]
    test_sample = test[model_features]
    # train the model
    # verbose = 1 gives the output of the training.
    print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
    start_time = datetime.datetime.now()
    X = train_sample.iloc[:,:-1]
    if use_classifier:
        Y = train_sample["target"].apply(np.sign)
        Y_test = test_sample["target"].apply(np.sign)
    else:
        # using risk adjusted return- continous value
        Y = train_sample["target"].apply(np.sign)
        Y_test = test_sample["target"].apply(np.sign)
    X_test = test_sample.iloc[:,:-1]
    # clean the data and nan values
    X = X.replace(np.nan, 0)
    Y = Y.replace(np.nan, 0)
    X_test = X_test.replace(np.nan, 0)
    Y_test = Y_test.replace(np.nan, 0)
    if use_classifier:
        RF = RandomForestClassifier(n_estimators = ntree, max_features = 5, verbose = 1)
        #clf = tree.DecisionTreeClassifier(max_leaf_nodes = 6, max_depth = 8)
    else:
        # ass in code for regressino classifier
        pass
    # clf = clf.fit(X, Y)
    RF.fit(X, Y)
    run_time = datetime.datetime.now() - start_time
    # run training on the test data
    results = RF.predict(X_test)
    # The % threshold needed to trigger a signal either way
    if use_binary and use_risk_adjusted:
        results = [signal(i, thold) for i in list(results)]
    acc_score = get_accuracy(Y_test, results)
    # This needs to change to handle the change in the target
    predictions = pd.DataFrame({"Date" : test['Date'],"Predictions": results})
    test_results = pd.merge(test, predictions,how= "left", on= "Date").fillna(0)
    # calculate the returns of the signal
    test_results["scaled_signal"] = test_results['Predictions'].shift(2).rolling(trade_horizon).sum()/trade_horizon
    # no shift needed as we have already done that in previous step
    test_results['strat_returns'] = test_results['logret']*test_results['scaled_signal']
    test_results['strat_returns_sum'] = test_results['strat_returns'].cumsum()
    strat_return = test_results['strat_returns'].sum()
    information_ratio = (test_results['strat_returns'].mean()*260)/(test_results['strat_returns'].std()*np.sqrt(260))
    
    # Store the data as needed
    performance_store['data_size'].append(data_size)
    performance_store['ntree'].append(ntree)
    performance_store['Accuracy_Score'].append(acc_score)
    performance_store['Info_Ratio'].append(information_ratio)
    performance_store['run_time'].append(run_time)
    performance_store['train_date_st'].append(train['Date'].iloc[0])
    performance_store['test_date_st'].append(test_results['Date'].iloc[0])
    performance_df = pd.DataFrame(performance_store)
    save_results = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\test_result_start%s_lkbk%s_ntree%s_thold%s.csv" % (start_row, ntree, ntree, thold)
    test_results.to_csv(save_results,index = False)

start_row 50000


Starting ML model now end date : 2007-12-27 07:00:00


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    1.0s finished


...

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.3s finished


start_row 150000


Starting ML model now end date : 2017-10-18 03:00:00


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   13.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.0s finished


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_functions import *
import datetime
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
data_file = pd.read_csv(r"C:\Users\edgil\Documents\Masters\dissertation\code64\data_set\CcyData.csv") # pd.read_csv(r"/storage/eurusd_train_normed.csv")
data_file = data_file.replace(np.nan, 0)
#eurusd_test = eurusd_test.replace(np.nan, 0)
performance_store = {"data_size" : [], "Accuracy_Score" : [], "ntree": [],
                     "Info_Ratio" : [], "run_time" : [], "train_date_st": [], "test_date_st": []}
########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
ntree = 15 # [21, 66]
trade_horizon = 24
model_features = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff",
                   "MF_ema_diff","LF_ema_diff", "LDN", "NY", "Asia", "target"]
test_buffer = 5
data_size = 50000
test_split = 0.75
total_data_needed = int(data_size*(1 + test_split)) + test_buffer
###### Set Targets ##############
trade_horizon = 24 # in hours
use_risk_adjusted = False # if True: training on the sharpe return else raw
data_file["target"] = calculate_target(data_file,trade_horizon,use_risk_adjusted)
################### Standardise Entire Dataset using rolling lookback windows ###############
features_to_standardise = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff",
                                "MF_ema_diff","LF_ema_diff"]
# roughly 3 yrs of data slightly less actually
window = 15000
data_normed = standardise_data(data_file,model_features ,features_to_standardise, window)
# add extra features non standardised
data_normed['Date'] = data_file['Date'].iloc[window:]
data_normed['CCY'] = data_file['CCY'].iloc[window:]
#data_normed = data_normed.replace(np.nan, 0)
start_row = 0
use_separated_chunk = False # Use a rollign window to train and test

In [14]:
trunc_data = data_file.loc[20000:,:]
test_buffer = 5
data_size = 5000
test_split = 0.75
train , test = create_train_test_file(trunc_data, data_size, test_split, test_buffer)
train_sample = train[model_features]
test_sample = test[model_features]
X = train_sample.iloc[:,:-1]
Y = train_sample["target"].apply(np.sign)
X_test = test_sample.iloc[:,:-1]
Y_test = test_sample["target"].apply(np.sign)
X = X.replace(np.nan, 0)
Y = Y.replace(np.nan, 0)
X_test = X_test.replace(np.nan, 0)
Y_test = Y_test.replace(np.nan, 0)
RF = RandomForestClassifier(n_estimators = 15, max_features = 5, verbose = 1)
RF.fit(X, Y)
clf = tree.DecisionTreeClassifier(max_leaf_nodes = 6, max_depth = 8)
clf = clf.fit(X, Y)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s finished


In [27]:
results = RF.predict(X_test)
print(get_accuracy(Y_test,results))
predictions = pd.DataFrame({"Date" : test['Date'],"Predictions": results})
print(predictions)

0.5428571428571428
                     Date  Predictions
0     2003-12-30 21:00:00          1.0
1     2003-12-30 22:00:00          1.0
2     2003-12-30 23:00:00          1.0
3     2003-12-31 00:00:00         -1.0
4     2003-12-31 01:00:00         -1.0
5     2003-12-31 02:00:00         -1.0
6     2003-12-31 03:00:00         -1.0
7     2003-12-31 04:00:00         -1.0
8     2003-12-31 05:00:00         -1.0
9     2003-12-31 06:00:00          1.0
10    2003-12-31 07:00:00         -1.0
11    2003-12-31 08:00:00         -1.0
12    2003-12-31 09:00:00         -1.0
13    2003-12-31 10:00:00         -1.0
14    2003-12-31 11:00:00         -1.0
15    2003-12-31 12:00:00         -1.0
16    2003-12-31 13:00:00         -1.0
17    2003-12-31 14:00:00         -1.0
18    2003-12-31 15:00:00         -1.0
19    2003-12-31 16:00:00         -1.0
20    2003-12-31 17:00:00         -1.0
21    2003-12-31 18:00:00          1.0
22    2003-12-31 19:00:00          1.0
23    2003-12-31 20:00:00          1.0
24    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished


In [ ]:
# train the model
# verbose = 1 gives the output of the training.
print("Starting ML model now end date : %s" % train['Date'].iloc[-1] )
start_time = datetime.datetime.now()
X = train_sample.iloc[:,:-1]
Y = train_sample["target"].apply(np.sign)
X_test = test_sample.iloc[:,:-1]
Y_test = test_sample["target"].apply(np.sign)
# clean the data and nan values
X = X.replace(np.nan, 0)
Y = Y.replace(np.nan, 0)
X_test = X_test.replace(np.nan, 0)
Y_test = Y_test.replace(np.nan, 0)
RF = RandomForestClassifier(n_estimators = ntree, max_features = 5, verbose = 1)
RF.fit(X, Y)
clf = tree.DecisionTreeClassifier(max_leaf_nodes = 6, max_depth = 8)
clf = clf.fit(X, Y)
run_time = datetime.datetime.now() - start_time
# run training on the test data
results = RF.predict(X_test)
# The % threshold needed to trigger a signal either way
if use_binary and use_risk_adjusted:
    thold = 0.55
    results = [signal(i, thold) for i in list(results)]
    acc_score = get_accuracy(Y_test, results)
# This needs to change to handle the change in the target
predictions = pd.DataFrame({"Date" : test['Date'],"Predictions": results})
test_results = pd.merge(test, predictions,how= "left", on= "Date").fillna(0)
# calculate the returns of the signal
test_results["scaled_signal"] = test_results['Predictions'].shift(2).rolling(trade_horizon).sum()/trade_horizon
# no shift needed as we have already done that in previous step
test_results['strat_returns'] = test_results['logret']*test_results['scaled_signal']
test_results['strat_returns_sum'] = test_results['strat_returns'].cumsum()
strat_return = test_results['strat_returns'].sum()
information_ratio = (test_results['strat_returns'].mean()*260)/(test_results['strat_returns'].std()*np.sqrt(260))

# Store the data as needed
performance_store['data_size'].append(data_size)
performance_store['ntree'].append(ntree)
performance_store['Accuracy_Score'].append(acc_score)
performance_store['Info_Ratio'].append(information_ratio)
performance_store['run_time'].append(run_time)
performance_store['train_date_st'].append(train['Date'].iloc[0])
performance_store['test_date_st'].append(test_results['Date'].iloc[0])
performance_df = pd.DataFrame(performance_store)
save_results = r"C:\Users\edgil\Documents\Masters\dissertation\code\dataset\test_result_start%s_lkbk%s_ntree%s_thold%s.csv" % (start_row, ntree, ntree, thold)
test_results.to_csv(save_results,index = False)